<p style="font-family: Arial; font-size:3em;color:black;"> Session 5 - Lab Exercise 9</p>

Students name in this Group:
- Gabriel Okazaki
- Julio Socher
- Kateryna Solonenko
- Mohit Sabharwal
- Riya Gupta
- Vaibhav Saini

In [61]:
# For this example, we will use K-Means Clustering Project database from Kaggle (https://www.kaggle.com/faressayah/k-means-clustering-private-vs-public-universities)
# We actually have the labels for this data set, but we will NOT use them for the KMeans clustering algorithm, since that is an unsupervised learning algorithm.
# As we will shortly see, we have a data frame with 777 observations on 18 variables.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [62]:
df = pd.read_csv('data/College_Data',index_col=0)
df.columns

Index(['Private', 'Apps', 'Accept', 'Enroll', 'Top10perc', 'Top25perc',
       'F.Undergrad', 'P.Undergrad', 'Outstate', 'Room.Board', 'Books',
       'Personal', 'PhD', 'Terminal', 'S.F.Ratio', 'perc.alumni', 'Expend',
       'Grad.Rate'],
      dtype='object')

In [63]:
df['Grad.Rate']['Cazenovia College'] = 100

# Try removing various columns (features) from the dataset and examin if it improves/degrades your K-Means model performance, or it may have little impact.
# Report 10 cases where you removed one or more features and indicate how it impacted the model performance.

/tmp/ipykernel_4423/3450590736.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Grad.Rate']['Cazenovia College'] = 100


In [64]:
df.head()

,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
Abilene Christian University,Yes,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
Adelphi University,Yes,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
Adrian College,Yes,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54
Agnes Scott College,Yes,417,349,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59
Alaska Pacific University,Yes,193,146,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15


In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 777 entries, Abilene Christian University to York College of Pennsylvania
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Private      777 non-null    object 
 1   Apps         777 non-null    int64  
 2   Accept       777 non-null    int64  
 3   Enroll       777 non-null    int64  
 4   Top10perc    777 non-null    int64  
 5   Top25perc    777 non-null    int64  
 6   F.Undergrad  777 non-null    int64  
 7   P.Undergrad  777 non-null    int64  
 8   Outstate     777 non-null    int64  
 9   Room.Board   777 non-null    int64  
 10  Books        777 non-null    int64  
 11  Personal     777 non-null    int64  
 12  PhD          777 non-null    int64  
 13  Terminal     777 non-null    int64  
 14  S.F.Ratio    777 non-null    float64
 15  perc.alumni  777 non-null    int64  
 16  Expend       777 non-null    int64  
 17  Grad.Rate    777 non-null    int64  
dtypes: 

In [66]:
from sklearn.cluster import KMeans

In [67]:
kmeans = KMeans(n_clusters=2, random_state=2)

In [68]:
kmeans.fit(df.drop('Private',axis=1))
Cluster_predict=kmeans.predict(df.drop('Private',axis=1))
df['Cluster_predict'] = Cluster_predict

/home/gabs/Projects/2024/george_brown/app_math/app_math_env/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [69]:
def convert_to_binary(cluster):  # converts labels for Private into binary values of 0 and 1
    if cluster=='Yes':
        return 1
    else:
        return 0

In [70]:
df['Cluster_actual'] = df['Private'].apply(convert_to_binary)

In [71]:
df['Cluster_actual'].value_counts()

Cluster_actual
1    565
0    212
Name: count, dtype: int64

In [72]:
df['Cluster_predict'].value_counts()

Cluster_predict
1    669
0    108
Name: count, dtype: int64

In [73]:
df['Cluster_predict'].value_counts()[0]

108

In [74]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [75]:
print('Baseline:')
print(confusion_matrix(df['Cluster_actual'],kmeans.labels_))
print(f"Accuracy score: {accuracy_score(df['Cluster_actual'], kmeans.labels_)}")

Baseline:
[[ 74 138]
 [ 34 531]]
Accuracy score: 0.7786357786357786


In [76]:
def try_model_with_dropped_feature(feature):
    df_copy = df.copy()
    kmeans = KMeans(n_clusters=2, n_init=10, random_state=2)
    print(f"Dropping feature: {feature}")
    kmeans.fit(df_copy.drop(['Private', feature],axis=1))
    Cluster_predict=kmeans.predict(df_copy.drop(['Private', feature],axis=1))
    df_copy['Cluster_predict'] = Cluster_predict

    # Invert cluster labels if 0 is predominant
    if df_copy['Cluster_predict'].value_counts()[1] < df_copy['Cluster_predict'].value_counts()[0]:
        df_copy['Cluster_predict'] = df_copy['Cluster_predict'].replace({0:1, 1:0})

    print(df_copy['Cluster_predict'].value_counts())
    print('Confusion matrix:')
    print(confusion_matrix(df['Cluster_actual'],kmeans.labels_))
    print(f"Accuracy score: {accuracy_score(df['Cluster_actual'], kmeans.labels_)}\n")

In [77]:
non_feature_colums = ['Private', 'Cluster_predict', 'Cluster_actual']
features_to_remove = [feature for feature in df.columns.tolist() if feature not in non_feature_colums]
features_to_remove

['Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F.Undergrad',
 'P.Undergrad',
 'Outstate',
 'Room.Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S.F.Ratio',
 'perc.alumni',
 'Expend',
 'Grad.Rate']

In [78]:
for feature in features_to_remove:
    try_model_with_dropped_feature(feature)

Dropping feature: Apps


Cluster_predict
1    625
0    152
Name: count, dtype: int64
Confusion matrix:
[[209   3]
 [416 149]]
Accuracy score: 0.46074646074646075

Dropping feature: Accept
Cluster_predict
1    674
0    103
Name: count, dtype: int64
Confusion matrix:
[[140  72]
 [534  31]]
Accuracy score: 0.22007722007722008

Dropping feature: Enroll
Cluster_predict
1    669
0    108
Name: count, dtype: int64
Confusion matrix:
[[ 74 138]
 [ 34 531]]
Accuracy score: 0.7786357786357786

Dropping feature: Top10perc
Cluster_predict
1    669
0    108
Name: count, dtype: int64
Confusion matrix:
[[ 74 138]
 [ 34 531]]
Accuracy score: 0.7786357786357786

Dropping feature: Top25perc
Cluster_predict
1    669
0    108
Name: count, dtype: int64
Confusion matrix:
[[ 74 138]
 [ 34 531]]
Accuracy score: 0.7786357786357786

Dropping feature: F.Undergrad
Cluster_predict
1    616
0    161
Name: count, dtype: int64
Confusion matrix:
[[189  23]
 [427 138]]
Accuracy score: 0.42084942084942084

Dropping feature: P.Undergrad
Cluster_p

Looking at the results of removing features individually, we had a loss of accuracy in the following cases:
- Apps
- Accept
- F.Undergrad
- Outstate
- Expend

These results sugest that these are the most important features for determining if a college is private or public